In [20]:
import numpy as np
import pandas as pd
import re,string
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
import pickle
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\hp
[nltk_data]     victus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
df=pd.read_csv('text.csv')
df=df.sample(frac=1,random_state=42)
print(df.head())
print(df.info())

        Unnamed: 0                                               text  label
36130        36130  id say maybe made them feel foolish but that w...      0
138065      138065  i joined the lds church i admit to feeling som...      0
146440      146440  i must admit i didnt feel like hugging him not...      3
103337      103337  i hate that i can still feel if any nerve is d...      0
315528      315528                  im actually feeling a little smug      1
<class 'pandas.core.frame.DataFrame'>
Index: 416809 entries, 36130 to 121958
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  416809 non-null  int64 
 1   text        416809 non-null  object
 2   label       416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 12.7+ MB
None


In [22]:
#dropping unnecessary column
df=df.drop(columns=['Unnamed: 0'])

In [24]:
#cleaning text using function
stop_words=set(stopwords.words('english'))
stemmer=PorterStemmer()
def clean_text(text):
    text=text.lower()
    text=re.sub(r"[^\w\s]","",text)
    text=" ".join(stemmer.stem(word)for word in text.split() if word not in stop_words)
    return text

df['text']=df['text'].apply(clean_text)

print(df.head())

                                                     text  label
36130   id say mayb made feel foolish would reeeeeeall...      0
138065  join ld church admit feel somewhat asham famil...      0
146440  must admit didnt feel like hug angri disgust d...      3
103337  hate still feel nerv damag badli enough often ...      0
315528                          im actual feel littl smug      1


In [25]:
#Training model
from sklearn.linear_model import LogisticRegression
log_regression=LogisticRegression()

vectorizer=TfidfVectorizer()
X=df['text']
Y=df['label']

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.15)

#creating pipeline
pipeline=Pipeline([('Vect',vectorizer),
                   ('chi',SelectKBest(chi2,k=2000)),
                   ('clf',LogisticRegression())])

#Training model
model=pipeline.fit(X_train,Y_train)

#Creating pickle file
with open('LogisticRegression2.pickle', 'wb') as f:
    pickle.dump(model, f)

c:\Users\hp victus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
#Test accuracy of logistic regression
from sklearn.metrics import accuracy_score
Y_test_pred=model.predict(X_test)
accuracy=accuracy_score(Y_test,Y_test_pred)
print(f'Accuracy of log_regression: {accuracy*100:.2f}')

Accuracy of log_regression: 88.11


In [27]:
#train accuracy of logistic regression
Y_test_pred=model.predict(X_train)
accuracy=accuracy_score(Y_train,Y_test_pred)
print(f'Accuracy of log_regression: {accuracy*100:.2f}')

Accuracy of log_regression: 88.40
